# Digital Video Stabilization

This notebook explores state of the art methods of video stabilization. It benchmarks these methods on a specific dataset of shaky videos (from tennis table matches and swimming competitions). Then, we propose some innovative ideas and compare them with these methods.

We will proceed as follows :


*   Video stabilization definition and examples ( real shaky + artificially generated distabilized videos).
*   State of the art methods.






In [39]:
!unzip samples.zip

Archive:  samples.zip
   creating: samples/
  inflating: samples/vibrate_randomly.mp4  
  inflating: samples/vibrate_up_down.mp4  
  inflating: samples/video_after_rotation.mp4  
  inflating: samples/video_after_translation.mp4  
  inflating: samples/videoplayback.mp4  
  inflating: samples/vibration.mp4   


In [41]:
!pip install vidstab
!apt install ffmpeg
import numpy as np
import cv2
import vidstab
import matplotlib.pyplot as plt
import imageio
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import os

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


# Generate destabilized videos

## Rotation

In [58]:
def rotate(frame, angle):
    cols, rows, _ = frame.shape
    rotate_around = (cols // 2, rows // 2)
    rot_mat = cv2.getRotationMatrix2D(rotate_around, angle, 1.0)
    result = cv2.warpAffine(frame, rot_mat, frame.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result


def rotate_video(video_path_in, video_name, rotation_increment, video_path_out):
    video = os.path.join(video_path_in, video_name)
    video = cv2.VideoCapture(video)
    fourcc = cv2.VideoWriter_fourcc("M", "J", "P", "G")
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)
    video_writer = cv2.VideoWriter(video_path_out, fourcc, fps, (width, height))
    while True:
        ret, frame = video.read()
        if not ret:
            break
        direction = np.random.choice([-1, 1])
        result = rotate(frame, rotation_increment * direction)
        rotation_increment = rotation_increment * direction
        video_writer.write(result)
    video_writer.release()
    video.release()

## Translation

In [57]:
def translate_vertically(frame, translation_rate):
    frame1 = np.zeros(frame.shape)
    h, w, _ = frame.shape

    if translation_rate > 0:
        frame1[translation_rate:h, :] = frame[: h - translation_rate, :]
    else:
        frame1[: h - translation_rate, :] = frame[translation_rate:h, :]
    return frame1


def translate_horizontally(frame, translation_rate):
    frame1 = np.zeros_like(frame)
    h, w, _ = frame.shape

    if translation_rate > 0:
        frame1[:, translation_rate:w] = frame[:, : w - translation_rate]
    else:
        frame1[:, : w - translation_rate] = frame[:, translation_rate:w]
    return frame1


def translate_video_horizontally(
    video_path_in, video_name, translation_rate, video_path_out
):
    video = os.path.join(video_path_in, video_name)

    video = cv2.VideoCapture(video)
    fourcc = cv2.VideoWriter_fourcc("M", "J", "P", "G")
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)
    video_writer = cv2.VideoWriter(video_path_out, fourcc, fps, (width, height))
    while True:
        ret, frame = video.read()
        if not ret:
            break
        result = translate_horizontally(frame, translation_rate)
        translation_rate += 5
        video_writer.write(result)
    video_writer.release()
    video.release()


def translate_video_vertically(
    video_path_in, video_name, translation_rate, video_path_out
):
    video = os.path.join(video_path_in, video_name)

    video = cv2.VideoCapture(video)
    fourcc = cv2.VideoWriter_fourcc("M", "J", "P", "G")
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)
    video_writer = cv2.VideoWriter(video_path_out, fourcc, fps, (width, height))
    while True:
        ret, frame = video.read()
        if not ret:
            break
        result = translate_vertically(frame, translation_rate)
        translation_rate += 5
        video_writer.write(result)
    video_writer.release()
    video.release()

## Destabilize with random translation

In [56]:
import numpy as np
from moviepy.editor import VideoClip


def shake_video_randomly(video_path_in, video_name, shake_intensity, video_path_out):
    video = os.path.join(video_path_in, video_name)
    video = cv2.VideoCapture(video)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)
    video_writer = cv2.VideoWriter(video_path_out, fourcc, fps, (width, height))
    while True:
        ret, frame = video.read()
        if not ret:
            break
        h, w, _ = frame.shape
        direction = np.random.choice(["up","left","right","down"])
        frame1 = np.zeros_like(frame)
        if direction == "up":
            frame1[: h - shake_intensity] = frame[shake_intensity:h]
        elif direction == "left":
            frame1[:, : w - shake_intensity] = frame[:, shake_intensity:w]
        elif direction == "down":
            frame1[shake_intensity:h] = frame[: h - shake_intensity]
        else:
            frame1[:, shake_intensity:w] = frame[:, : w - shake_intensity]
        video_writer.write(frame1)
    video_writer.release()
    video.release()

# Video Stabilization Using Point Feature Matching in OpenCV

In [55]:
from Point_Feature_Matching_opencv import stabilize_point_feature_match
videos=os.listdir("./samples")
for video in videos:
  input_file=os.path.join("./samples",video)
  output_file=os.path.join("./opencv",video.replace(".mp4","_stabilized.mp4"))
  stabilize_point_feature_match(input_file,output_file)

Frame: 0/100 -  Tracked points : 200
Frame: 1/100 -  Tracked points : 200
Frame: 2/100 -  Tracked points : 189
Frame: 3/100 -  Tracked points : 200
Frame: 4/100 -  Tracked points : 174
Frame: 5/100 -  Tracked points : 200
Frame: 6/100 -  Tracked points : 188
Frame: 7/100 -  Tracked points : 176
Frame: 8/100 -  Tracked points : 200
Frame: 9/100 -  Tracked points : 200
Frame: 10/100 -  Tracked points : 188
Frame: 11/100 -  Tracked points : 200
Frame: 12/100 -  Tracked points : 184
Frame: 13/100 -  Tracked points : 187
Frame: 14/100 -  Tracked points : 200
Frame: 15/100 -  Tracked points : 200
Frame: 16/100 -  Tracked points : 200
Frame: 17/100 -  Tracked points : 200
Frame: 18/100 -  Tracked points : 200
Frame: 19/100 -  Tracked points : 200
Frame: 20/100 -  Tracked points : 200
Frame: 21/100 -  Tracked points : 200
Frame: 22/100 -  Tracked points : 200
Frame: 23/100 -  Tracked points : 200
Frame: 24/100 -  Tracked points : 200
Frame: 25/100 -  Tracked points : 183
Frame: 26/100 -  Track

# Gaussian kernel low-pass filter on camera motion (ffmpeg)

In [44]:
import subprocess
videos=os.listdir("./samples")
transform_file = "transform_vectors.trf"
for video in videos:
  input_file=os.path.join("./samples",video)
  output_file=os.path.join("./ffmpeg",video.replace(".mp4","_stabilized.mp4"))
  command = ["ffmpeg", "-i", input_file, "-vf", f"vidstabdetect=stepsize=6:shakiness=8:accuracy=9:result={transform_file}", "-f", "null", "-"]
  subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  command = ["ffmpeg", "-i", input_file, "-vf", f"vidstabtransform=input={transform_file}", "-c:a", "copy", output_file]
  subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Stabilization using vidstab package

In [45]:
from vidstab import VidStab
import matplotlib.pyplot as plt
videos=os.listdir("./samples")
for video in videos:
  input_file=os.path.join("./samples",video)
  output_file=os.path.join("./vidstab",video.replace(".mp4","_stabilized.mp4"))
  stabilizer = VidStab()
  stabilizer.stabilize(input_path=input_file, output_path=output_file)


The results are stored in the folder results (https://github.com/centralelyon/video-stabilization/tree/main/results).